# Assignment 2

## Preliminary Steps

Let's import all the needed packages

In [42]:
# Handle files and unzip
import os
import requests
import zipfile

import re

import pandas as pd
import numpy as np

# Neural Networks
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Word tokenization
from tensorflow.keras.preprocessing.text import Tokenizer


import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
!pip install tqdm
from tqdm import tqdm as tq 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import tensorflow as tf 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Let's download the dataset

In [43]:
def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

Look inside our dataset creating a first dataframe reading the `test_pairs.csv` file

In [44]:
test_path = os.path.join(os.getcwd(), 'dataset', 'train_pairs.csv')



## Data preparation

As stated in [specifications.ipynb](specifications.ipynb), each dataset sample is comprised of:

*     A claim to verify
*     A set of semantically related statements (evidence set)
*     Fact checking label: either evidences support or refute the claim.

Handling the evidence set from the point of view of neural models may imply some additional complexity: if the evidence set is comprised of several sentences we might incur in memory problems.

To this end, we further simplify the problem by building (claim, evidence) pairs. The fact checking label is propagated as well.

Example:

     Claim: c1 
     Evidence set: [e1, e2, e3]
     Label: S (support)

--->

    (c1, e1, S),
    (c1, e2, S),
    (c1, e3, S)

So now we construct a new dataframe where each sample is in the form (Claim, Evidence, Lable).

In [45]:
# the evidences seems to be separated by a tab (\t) 
# so it may be used to split the Evidence string

# this dataset is a mess
dataset_path = os.path.join(os.getcwd(), 'dataset', 'train_pairs.csv')

def sentence_cleaning(sentence: str) -> str:
    """
    cleans up a sentence in the dataset using regular expressions
    :param sentence: the sentence to clean-up
    :return
        - string cleaned
    """
    
    # removes "-LRB-" and "-RRB-" strings and commas
    sentence = re.sub("-LRB-","", sentence)
    sentence = re.sub("-RRB-", "", sentence)
    #sentence = re.sub(",", "", sentence)
    sentence = re.sub("-LSB-", "", sentence)
    sentence = re.sub("-RSB-", "", sentence)
    
    return sentence
    

def format_dataset(dataset: str, debug: bool = True) -> pd.DataFrame:
    """
    Reads out the csv file and returns a dataframe with 
    {Claim, Evidence, Label} row
    
    :param dataset: dataset csv file path
    :param debug: if True prints out data for debugging purposes
    
    :return
        - dataframe with (claim, evidence, label) rows
    """
    
    df = pd.read_csv(dataset)
    dataframe_rows = []
    df_size = df.shape[0]

    for i in range(0, df_size):

        claim = df["Claim"][i]
        claim = sentence_cleaning(str(claim))

        label = df["Label"][i]

        ev_list = df["Evidence"][i].split('\t')
        evidence = sentence_cleaning(str(ev_list[1]))
        #evidence = ev_list[1]

        

        # create single dataframe row
        dataframe_row = {
            "Claim": claim,
            "Evidence": evidence,
            "Label": label
        }

        if debug: 
            print(claim)
            print(evidence)
            print(label)

        dataframe_rows.append(dataframe_row)

    df = pd.DataFrame(dataframe_rows)
    
    return df, dataframe_rows

df, df_rows = format_dataset(dataset_path, False)

df['Claim_Evidence'] = df.Claim + df.Evidence
df['Label'] = df.Label

In [46]:
df.Claim_Evidence[50]

"Jules Dumont d'Urville was born on May 23, 1790.Jules Sébastien César Dumont d'Urville  23 May 1790 -- 8 May 1842  was a French explorer , naval officer and rear admiral , who explored the south and western Pacific , Australia , New Zealand and Antarctica ."

Let's see how the resulting dataframe looks like

In [47]:
df.head()

,Claim,Evidence,Label,Claim_Evidence
0,Chris Hemsworth appeared in A Perfect Getaway.,Hemsworth has also appeared in the science fic...,SUPPORTS,Chris Hemsworth appeared in A Perfect Getaway....
1,Roald Dahl is a writer.,"Roald Dahl langpronˈroʊ.əld _ ˈdɑːl , ˈɾuː...",SUPPORTS,Roald Dahl is a writer.Roald Dahl langpronˈr...
2,Roald Dahl is a governor.,"Roald Dahl langpronˈroʊ.əld _ ˈdɑːl , ˈɾuː...",REFUTES,Roald Dahl is a governor.Roald Dahl langpron...
3,Ireland has relatively low-lying mountains.,The island 's geography comprises relatively l...,SUPPORTS,Ireland has relatively low-lying mountains.The...
4,Ireland does not have relatively low-lying mou...,The island 's geography comprises relatively l...,REFUTES,Ireland does not have relatively low-lying mou...


In [48]:
y_train = df.Label

Turning claims and Evidences into sequences of integers

In [49]:
def make_sequences(texts, lower=True, filters='!"#$%&()*+-/:;<=>?@[\\]^_`{|}~\t\n'):
    """
    Turn a set of texts into sequences of integers
    
    :param texts: the set of texts to turn into sequences
    :param lower: boolean. Whether to convert the texts to lowercase
    :param filters: a string where each element is a character that will be filtered from the texts
    """
    
    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(texts)
    
    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts
    
    # Convert text to sequences of integers
    sequences = tokenizer.texts_to_sequences(texts)
    
    return word_idx, idx_word, num_words, word_counts, sequences

def encode_dataframe(dataframe):
    """
    Creates sequences of integers for both Claim and Evidence columns
    """
    
    # creates lists of claims and evidences
    #claims = list(dataframe['Claim'])
    #evidences = list(dataframe['Evidence'])
    claims_evidences = list(dataframe['Claim_Evidence'])
    
    cl_word_idx, cl_idx_word, cl_num_words, cl_word_counts, seq_claim_ev = make_sequences(claims_evidences)
    #ev_word_idx, ev_idx_word, ev_num_words, ev_word_counts, seq_evidences = make_sequences(evidences)

    
    return seq_claim_ev

#seq_claims_evidences = encode_dataframe(df)

In [50]:
lower=True 
filters='!"#$%&()*+-/:;<=>?@[\\]^_`{|}~\t\n'
# Create the tokenizer object and train on texts
tokenizer = Tokenizer(lower=lower, filters=filters)
tokenizer.fit_on_texts(df.Claim_Evidence)
# Create look-up dictionaries and reverse look-ups
word_idx = tokenizer.word_index
idx_word = tokenizer.index_word
num_words = len(word_idx) + 1
word_counts = tokenizer.word_counts
# Convert text to sequences of integers
token_claim = tokenizer.texts_to_sequences(df.Claim)
token_evidence = tokenizer.texts_to_sequences(df.Evidence)

Constructing the input matrices for the embedding step. We construct the matrices starting from `seq_claims` and `seq_evidences`. The number of rows of the matrices is the number of elements in `seq_claims` and `seq_evidences`, while the number of columns is the size of the longest sequences. The rows corresponding to shorter sequences are filled with zero-padding.

In [51]:
# computes the longest token sequence
def longest_seq(seq):
    seq_len = [len(i) for i in seq]
    
    return max(seq_len)

def matrix_from_sequences(sequences):
    """
    Builds a matrix of shape [batch_size, max_tokens]
    """
    
    max_tokens = longest_seq(sequences)
    seq_length = len(sequences)
    
    matrix = np.zeros((len(sequences), max_tokens), dtype=np.int32)
    
    for i in range(0, seq_length):
        for j in range(0, len(sequences[i])):
            matrix[i][j] = sequences[i][j]
    
    return matrix

claim_matrix = matrix_from_sequences(token_claim)
evidence_matrix = matrix_from_sequences(token_evidence)

print(claim_matrix.shape)
print(evidence_matrix.shape)


(121740, 76)
(121740, 138)


I dunno if this is correct

### Sentence Embedding

Let's reshape the claim and evidence matrixes into [batch_size, sequence_length, embedding_dim]. The embedding is created with keras, involving the similarity of the phrases. Check on https://keras.io/api/layers/core_layers/embedding/

Try to follow the first approach, i donnot know if it is right

In [52]:
def sentence_embedding(matrix, input_dim, output_dim, input_length):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Embedding(input_dim = input_dim, output_dim = output_dim, input_length = input_length))

  model.compile('rmsprop', 'mse')
  reshaped_matrix = model.predict(matrix)
  return reshaped_matrix, model

In [53]:
reshaped_claim_matrix , embed_claim = sentence_embedding(claim_matrix, 121740, 50, 76)

In [54]:
reshaped_claim_matrix = []

In [55]:
reshaped_evidence_matrix , embed_evidence = sentence_embedding(evidence_matrix, 121740, 50, 138)

In [56]:
#matrix_concat = np.concatenate((reshaped_claim_matrix,reshaped_evidence_matrix),axis=1)

In [57]:
reshaped_evidence_matrix = []

In [60]:
embed_claim.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 76, 50)            6087000   
                                                                 
Total params: 6,087,000
Trainable params: 6,087,000
Non-trainable params: 0
_________________________________________________________________


Summary of the model

In [61]:
embedding_claim = embed_claim.get_layer('embedding_4').get_weights()[0]
embedding_evidence = embed_evidence.get_layer('embedding_5').get_weights()[0]

In [62]:
embedding_claim.shape

(121740, 50)

In [ ]:
#embedding_claim = list(embedding_claim)

In [63]:
embed_concat = np.concatenate((embedding_claim,embedding_evidence),axis=1)

In [64]:
embed_concat.shape

(121740, 100)

In [65]:
X_train = np.concatenate((claim_matrix, evidence_matrix),axis=1)

In [66]:
X_train.shape

(121740, 214)

In [68]:
tokenizer_y = Tokenizer()
tokenizer_y.fit_on_texts(y_train)
y_train = tokenizer_y.texts_to_sequences(y_train)

In [69]:
y_train = tf.keras.utils.to_categorical( y_train, num_classes=None, dtype='int')

In [70]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *
from keras.utils.np_utils import to_categorical
from keras.initializers import Constant

In [73]:
num_words

87727

In [78]:
embedding_dim = 100
embedding_matrix = embed_concat
model = Sequential()
sequence_length = X_train.shape[1]
model.add(Embedding(121740,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=sequence_length,
                    trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True)))
model.add(Bidirectional(CuDNNLSTM(32)))
model.add(Dropout(0.25))
model.add(Dense(units=3, activation='softmax', use_bias=False))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
batch_size = 128
print(model.summary())

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 214, 100)          12174000  
                                                                 
 spatial_dropout1d_2 (Spatia  (None, 214, 100)         0         
 lDropout1D)                                                     
                                                                 
 bidirectional_4 (Bidirectio  (None, 214, 128)         84992     
 nal)                                                            
                                                                 
 bidirectional_5 (Bidirectio  (None, 64)               41472     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                      

In [79]:
history = model.fit(X_train, y_train,epochs=5, batch_size=batch_size, verbose=1)

Epoch 1/5
 55/952 [>.............................] - ETA: 13:07 - loss: 0.6522 - accuracy: 0.7259

KeyboardInterrupt: ignored

In [ ]:
preds = model.predict([X_train], batch_size=batch_size, verbose=1).astype(int)
print(preds)